In [ ]:
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from dash import no_update


# Create a dash application
app = dash.Dash(__name__)

#read the data
launch_site = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
min_payload = launch_site['Payload Mass (kg)'].min()
max_payload = launch_site['Payload Mass (kg)'].max()

app.layout = html.Div(children=[ html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                
                                
                                dcc.Dropdown(id='site-dropdown',
                                                    options=[
                                                        {'label':'All Sites', 'value':'ALL'},
                                                        {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                                        {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                                        {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                                        {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'}
                                                    ],
                                                    value='ALL',
                                                    placeholder='Select a Launch Site here',
                                                    style={'width':'80%', 'padding':'3px', 'font-size':'20px','text-align-last':'center'},
                                                    searchable=True
                                                    ), 
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                
                                html.P('Payload Mass (KG):'),
                                
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000,
                                              marks={0:'0', 100:'100'},
                                              value=[min_payload, max_payload]),
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])



@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
             Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = launch_site
    if entered_site == "ALL":
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Success count for all launch sites')
        return fig
    else:
        filtered_df = launch_site[launch_site['Launch Site'] == entered_site]
        filtered_df = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        fig = px.pie(filtered_df, values='class count',names='class',title=f"Total Success Launches for site {entered_site}")
        return fig

@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
             Input(component_id='site-dropdown', component_property='value'),
             Input(component_id='payload-slider', component_property='value'))    
def scatter_chart(entered_site,payload):
    filtered_df = launch_site[launch_site['Payload Mass (kg)'].between(payload[0],payload[1])]
    # thought reusing filtered_df may cause issues, but tried it out of curiosity and it seems to be working fine
    
    if entered_site=='ALL':
        fig=px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Success count on Payload mass for all sites')
        return fig
    else:
        fig=px.scatter(filtered_df[filtered_df['Launch Site']==entered_site],x='Payload Mass (kg)',y='class',color='Booster Version Category',title=f"Success count on Payload mass for site {entered_site}")
        return fig

if __name__ == '__main__':
    # REVIEW8: Adding dev_tools_ui=False, dev_tools_props_check=False can prevent error appearing before calling callback function
    app.run_server()
    
#whenever writing a dash code there must be somethings that should be included
#load the data, start the app, app layout(html.Div(children=)) the layout of the page., must include dcc.graph to display the charts on the dashboard,
#include a @call back function that is output and input and then define the function, run the app.

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
